# SVD ALGORITHM SCIPY

In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

## data 1 soft :

In [17]:
df =pd.read_csv(r"C:\Users\hp\Downloads\prepro19.csv")

## Checking the density of the rating matrix

In [23]:
#la creation de la matrice de rating :
final_ratings_matrix = df.pivot_table(index = 'User_ID', columns ='Product_ID', values = 'Rating').fillna(0)
print('Shape de la matrice_rating est : ', final_ratings_matrix.shape)

#la recherche des colonnes non nulles dans la matrice rating
num_of_ratings = np.count_nonzero(final_ratings_matrix)
print('num de ratings = ', num_of_ratings)

#Trouver le nombre possible de notes en fonction du nombre d'utilisateurs et de produits.
possible_num_of_ratings = final_ratings_matrix.shape[0] * final_ratings_matrix.shape[1]
print('le nombre possible de rating est = ', possible_num_of_ratings)

#la densité des rating
density = (num_of_ratings/possible_num_of_ratings)
density *= 100
print ('density: {:4.2f}%'.format(density))

final_ratings_matrix.head()


Shape de la matrice_rating est :  (19681, 1381)
num de ratings =  20000
le nombre possible de rating est =  27179461
density: 0.07%


Product_ID,0615179088,0763855553,0842340351,1413313701,1426296355,1600775411,1935341049,3959828160,3959828233,3959828241,...,B01ETNVTSK,B01EU8RG7M,B01F4PYHX6,B01F70HGA4,B01F70HHBC,B01F7RJHIQ,B01H39M7ME,B01HAP3NJM,B01HAP3TX2,B01HAP47PQ
User_ID,,,,,,,,,,,,,,,,,,,,,
A0315022OKCLO1GNGIWT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0724185119EU7PTJSQT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0735469C3RVU9AWMDCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0798330IOVQKXNRRE5Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0823528HTX8UO6IPJ4V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
def calculate_sparsity(matrix):
    total_elements = matrix.size
    zero_elements = np.count_nonzero(matrix == 0)
    sparsity = zero_elements / total_elements
    return sparsity

 ==> Une densité de 0.07% signifie que seulement 0.07% de la matrice de notation est remplie de valeurs non nulles. En d'autres termes, la grande majorité des entrées de la matrice sont des valeurs manquantes ou nulles. 

#### donc c'est un matrice sparce !!

## CSR matrix

CSR (Compressed Sparse Row) est un format de représentation des matrices sparse (matrices contenant principalement des zéros)

In [26]:
# la transformation : 
final_ratings_sparse = csr_matrix(final_ratings_matrix.values)

## SVD algorithm

svds pour les matrices sparces

In [27]:
U, S, Vt = svds(final_ratings_sparse, k = 3) 

In [29]:
# la transformation de S en une table :
sigma = np.diag(S)

## la prédiction des rating :

In [30]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt) 

# la prédiction :
preds_df = pd.DataFrame(abs(predicted_ratings), columns = final_ratings_matrix.columns)
preds_matrix = csr_matrix(preds_df.values)

In [31]:
preds_df.head()

Product_ID,0615179088,0763855553,0842340351,1413313701,1426296355,1600775411,1935341049,3959828160,3959828233,3959828241,...,B01ETNVTSK,B01EU8RG7M,B01F4PYHX6,B01F70HGA4,B01F70HHBC,B01F7RJHIQ,B01H39M7ME,B01HAP3NJM,B01HAP3TX2,B01HAP47PQ
0,1.065065e-34,6.107553e-33,2.222179e-32,2.983428e-23,3.516778e-19,2.633485e-21,3.148285e-31,7.283328e-32,7.404676e-32,6.517025e-33,...,1.045527e-31,7.855108e-31,2.251273e-30,2.216484e-31,9.721058e-32,7.607196e-17,2.175016e-32,7.190444e-32,3.525186e-31,5.420072e-31
1,7.696402e-34,9.402471e-34,2.092981e-36,2.810221e-27,3.312948e-23,2.480833e-25,8.438717e-34,8.863546e-34,7.752427e-34,3.720994e-35,...,2.727699e-34,5.309109e-34,4.642726e-33,1.971810e-34,6.633356e-34,6.372663e-21,2.240523e-34,4.089374e-34,1.367882e-33,2.383540e-34
2,1.308599e-34,9.625937e-38,7.332498e-36,4.729902e-28,5.572365e-24,4.180792e-26,1.554975e-34,1.918927e-34,1.600386e-34,1.147755e-36,...,3.537216e-35,2.714109e-34,8.784819e-34,7.871708e-35,1.316575e-34,1.203600e-21,4.472579e-35,8.377495e-35,2.678047e-34,2.077772e-34
3,9.709166e-34,3.458001e-34,3.610814e-35,7.862964e-28,9.286214e-24,6.911658e-26,1.104107e-33,1.329216e-33,1.120815e-33,1.922742e-35,...,2.784023e-34,1.563032e-33,6.494260e-33,4.696743e-34,9.287304e-34,1.726164e-21,3.158732e-34,5.853705e-34,1.937269e-33,1.139217e-33
4,1.847909e-34,4.479110e-35,2.158338e-35,1.815444e-26,2.139955e-22,1.602565e-24,4.035282e-34,3.035939e-34,2.626767e-34,6.823200e-36,...,1.148960e-34,8.034619e-34,1.207320e-34,2.312450e-34,2.387811e-34,4.632229e-20,7.436067e-35,1.573365e-34,1.594932e-34,5.724833e-34


==> Les valeurs de notation sont très petites en raison de la nature "sparse" de la matrice

## la fonction de recommandation

In [32]:
def recommend_items(user_index, interactions_matrix, preds_matrix, num_recommendations):
    
    # determination des deux matrices predict et réelle
    user_ratings = interactions_matrix[user_index,:].toarray().reshape(-1)
    user_predictions = preds_matrix[user_index,:].toarray().reshape(-1)

    #la ceaction de dataframe avec les deux matrices 
    temp = pd.DataFrame({'user_ratings': user_ratings, 'user_predictions': user_predictions})
    temp['Recommended Products'] = np.arange(len(user_ratings))
    temp = temp.set_index('Recommended Products')
    
    #le filltrage des rating qui sonts nulle dans la matrice réelle
    temp = temp.loc[temp.user_ratings == 0]   
    
    # la recommandation des produits with top rating
    temp = temp.sort_values('user_predictions',ascending=False)
    print('\nla recommandations de user (user_id = {}):\n'.format(user_index))
    print(temp['user_predictions'].head(num_recommendations))

 ### Exemple de recommandation avec user = 100 et 35

In [33]:
recommend_items(100,final_ratings_sparse,preds_matrix,10)


la recommandations de user (user_id = 100):

Recommended Products
1117    2.212893e-12
879     1.717945e-14
1087    8.364242e-15
829     8.110248e-15
1376    3.356728e-15
1125    1.652239e-15
1348    1.603165e-15
719     5.372209e-16
1095    4.142261e-16
1335    3.865748e-17
Name: user_predictions, dtype: float64


#### underfitting !!!!!!!!!

## Evaluation de modéle

In [34]:
final_ratings_matrix['user_index'] = np.arange(0, final_ratings_matrix.shape[0])
final_ratings_matrix.set_index(['user_index'], inplace=True)

# Actual ratings given by users
final_ratings_matrix.head()

Product_ID,0615179088,0763855553,0842340351,1413313701,1426296355,1600775411,1935341049,3959828160,3959828233,3959828241,...,B01ETNVTSK,B01EU8RG7M,B01F4PYHX6,B01F70HGA4,B01F70HHBC,B01F7RJHIQ,B01H39M7ME,B01HAP3NJM,B01HAP3TX2,B01HAP47PQ
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
average_rating = final_ratings_matrix.mean()
average_rating.head()

Product_ID
0615179088    0.001728
0763855553    0.013770
0842340351    0.004471
1413313701    0.001575
1426296355    0.001474
dtype: float64

In [36]:
preds_df.head()

Product_ID,0615179088,0763855553,0842340351,1413313701,1426296355,1600775411,1935341049,3959828160,3959828233,3959828241,...,B01ETNVTSK,B01EU8RG7M,B01F4PYHX6,B01F70HGA4,B01F70HHBC,B01F7RJHIQ,B01H39M7ME,B01HAP3NJM,B01HAP3TX2,B01HAP47PQ
0,1.065065e-34,6.107553e-33,2.222179e-32,2.983428e-23,3.516778e-19,2.633485e-21,3.148285e-31,7.283328e-32,7.404676e-32,6.517025e-33,...,1.045527e-31,7.855108e-31,2.251273e-30,2.216484e-31,9.721058e-32,7.607196e-17,2.175016e-32,7.190444e-32,3.525186e-31,5.420072e-31
1,7.696402e-34,9.402471e-34,2.092981e-36,2.810221e-27,3.312948e-23,2.480833e-25,8.438717e-34,8.863546e-34,7.752427e-34,3.720994e-35,...,2.727699e-34,5.309109e-34,4.642726e-33,1.971810e-34,6.633356e-34,6.372663e-21,2.240523e-34,4.089374e-34,1.367882e-33,2.383540e-34
2,1.308599e-34,9.625937e-38,7.332498e-36,4.729902e-28,5.572365e-24,4.180792e-26,1.554975e-34,1.918927e-34,1.600386e-34,1.147755e-36,...,3.537216e-35,2.714109e-34,8.784819e-34,7.871708e-35,1.316575e-34,1.203600e-21,4.472579e-35,8.377495e-35,2.678047e-34,2.077772e-34
3,9.709166e-34,3.458001e-34,3.610814e-35,7.862964e-28,9.286214e-24,6.911658e-26,1.104107e-33,1.329216e-33,1.120815e-33,1.922742e-35,...,2.784023e-34,1.563032e-33,6.494260e-33,4.696743e-34,9.287304e-34,1.726164e-21,3.158732e-34,5.853705e-34,1.937269e-33,1.139217e-33
4,1.847909e-34,4.479110e-35,2.158338e-35,1.815444e-26,2.139955e-22,1.602565e-24,4.035282e-34,3.035939e-34,2.626767e-34,6.823200e-36,...,1.148960e-34,8.034619e-34,1.207320e-34,2.312450e-34,2.387811e-34,4.632229e-20,7.436067e-35,1.573365e-34,1.594932e-34,5.724833e-34


In [37]:
avg_preds=preds_df.mean()
avg_preds.head()

Product_ID
0615179088    1.058666e-18
0763855553    6.308230e-19
0842340351    6.006300e-20
1413313701    4.036416e-11
1426296355    4.757857e-07
dtype: float64

In [38]:
rmse_df = pd.concat([average_rating, avg_preds], axis=1)

rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']

rmse_df.head()

,Avg_actual_ratings,Avg_predicted_ratings
Product_ID,,
0615179088,0.001728,1.058666e-18
0763855553,0.013770,6.308230e-19
0842340351,0.004471,6.006300e-20
1413313701,0.001575,4.036416e-11
1426296355,0.001474,4.757857e-07


In [142]:
RMSE=mean_squared_error(rmse_df['Avg_actual_ratings'], rmse_df['Avg_predicted_ratings'], squared=False)
print(f'RMSE SVD Model = {RMSE} \n')
print('sparsity 1',calculate_sparsity(final_ratings_matrix))

RMSE SVD Model = 0.004789440153431951 

sparsity 1 0.9992641502346202


# data 2 electronics :

In [97]:
df1 =pd.read_csv(r"C:\Users\hp\Downloads\Electronics.csv")

In [98]:
df1

,Unnamed: 0,User_ID,Product_ID,Rating
0,6655415,A3GX0FAMEXV6FB,B009M0TFFU,5.0
1,7367340,A1CPRP3VFJRS1R,B00D78Q2AO,4.0
2,1501733,A11T807LX2EF00,B000WXQT9Q,5.0
3,1411577,A680RUE1FDO8B,B000UY3NR8,5.0
4,3201819,A3S5WYWYKSAI9C,B003BEDTBE,3.0
...,...,...,...,...
19995,5152501,A25UZ7MA72SMKM,B005QCDY50,5.0
19996,5304792,AAXAKFQEAQPWC,B0064SZ5U2,5.0
19997,6187193,A25C2M3QF9G7OQ,B008DHS1K8,4.0
19998,5033176,AYOMAHLWRQHUG,B005K2X8L6,5.0


In [99]:
#la creation de la matrice de rating :
final_ratings_matrix1 = df1.pivot_table(index = 'User_ID', columns ='Product_ID', values = 'Rating').fillna(0)
print('Shape de la matrice_rating est : ', final_ratings_matrix1.shape)

#la recherche des colonnes non nulles dans la matrice rating
num_of_ratings1 = np.count_nonzero(final_ratings_matrix1)
print('num de ratings = ', num_of_ratings1)

#Trouver le nombre possible de notes en fonction du nombre d'utilisateurs et de produits.
possible_num_of_ratings1 = final_ratings_matrix1.shape[0] * final_ratings_matrix1.shape[1]
print('le nombre possible de rating est = ', possible_num_of_ratings1)

#la densité des rating
density1 = (num_of_ratings1/possible_num_of_ratings1)
density1 *= 100
print ('density: {:4.2f}%'.format(density1))

final_ratings_matrix1.head()

Shape de la matrice_rating est :  (1540, 13444)
num de ratings =  20000
le nombre possible de rating est =  20703760
density: 0.10%


Product_ID,0972683275,1400501466,1400501776,1400532655,140053271X,7562434166,9043413585,9800359788,9981739588,998376234X,...,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00L403O94,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8
User_ID,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105S56ODHGJEK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105TOJ6LTVMBG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10AFVU66A79Y1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## CSR matrix

In [100]:
# la transformation : 
final_ratings_sparse1 = csr_matrix(final_ratings_matrix1.values)

## SVD ALGO

In [101]:
# determiner les trois matrices de SVD :
U1, S1, Vt1 = svds(final_ratings_sparse1, k = 50) 

In [102]:
# la transformation de S en une table :
sigma1 = np.diag(S1)

## la prédiction des rating :


In [103]:
predicted_ratings1 = np.dot(np.dot(U1, sigma1), Vt1) 

# la prédiction :
preds_df1 = pd.DataFrame(abs(predicted_ratings1), columns = final_ratings_matrix1.columns)
preds_matrix1 = csr_matrix(preds_df1.values)

In [104]:
preds_df1.head()

Product_ID,0972683275,1400501466,1400501776,1400532655,140053271X,7562434166,9043413585,9800359788,9981739588,998376234X,...,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00L403O94,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8
0,0.005615,0.001589,0.001616,0.000068,0.000026,0.001323,0.000320,0.016655,0.000962,0.002617,...,0.000937,0.023436,0.013831,0.050996,0.093397,0.004688,0.014737,0.024711,0.100877,0.015568
1,0.000373,0.002407,0.008561,0.002617,0.000035,0.006425,0.007553,0.013065,0.001476,0.000005,...,0.031365,0.042511,0.000215,0.002640,0.025026,0.019151,0.001955,0.057128,0.046192,0.039280
2,0.000187,0.000243,0.000106,0.000641,0.000067,0.000217,0.000264,0.000973,0.000602,0.001718,...,0.007571,0.003695,0.001559,0.006632,0.007211,0.125109,0.008773,0.002659,0.001556,0.012411
3,0.000021,0.000376,0.000274,0.000276,0.000014,0.002255,0.003132,0.003579,0.000144,0.000192,...,0.001373,0.009667,0.000364,0.008091,0.000967,0.010391,0.003675,0.006302,0.006837,0.005545
4,0.001208,0.000810,0.000083,0.002908,0.000004,0.005848,0.010397,0.007151,0.000333,0.002449,...,0.003531,0.024186,0.001148,0.005435,0.017878,0.028179,0.008829,0.005546,0.001331,0.005982


## Exemple de recommandation avec user = 1 et 2

In [105]:
recommend_items(100,final_ratings_sparse1,preds_matrix1,10)


la recommandations de user (user_id = 100):

Recommended Products
10511    0.649557
11949    0.599119
13329    0.419783
6117     0.419261
5817     0.371202
8328     0.363029
13280    0.355840
5725     0.336752
12406    0.325128
11697    0.318952
Name: user_predictions, dtype: float64


## Evaluation de modéle

In [106]:
final_ratings_matrix1['user_index'] = np.arange(0, final_ratings_matrix1.shape[0])
final_ratings_matrix1.set_index(['user_index'], inplace=True)

# Actual ratings given by users
final_ratings_matrix1.head()

Product_ID,0972683275,1400501466,1400501776,1400532655,140053271X,7562434166,9043413585,9800359788,9981739588,998376234X,...,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00L403O94,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
average_rating1 = final_ratings_matrix1.mean()
average_rating1.head()

Product_ID
0972683275    0.003247
1400501466    0.003247
1400501776    0.003247
1400532655    0.005844
140053271X    0.000649
dtype: float64

In [108]:
preds_df1.head()

Product_ID,0972683275,1400501466,1400501776,1400532655,140053271X,7562434166,9043413585,9800359788,9981739588,998376234X,...,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00L403O94,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8
0,0.005615,0.001589,0.001616,0.000068,0.000026,0.001323,0.000320,0.016655,0.000962,0.002617,...,0.000937,0.023436,0.013831,0.050996,0.093397,0.004688,0.014737,0.024711,0.100877,0.015568
1,0.000373,0.002407,0.008561,0.002617,0.000035,0.006425,0.007553,0.013065,0.001476,0.000005,...,0.031365,0.042511,0.000215,0.002640,0.025026,0.019151,0.001955,0.057128,0.046192,0.039280
2,0.000187,0.000243,0.000106,0.000641,0.000067,0.000217,0.000264,0.000973,0.000602,0.001718,...,0.007571,0.003695,0.001559,0.006632,0.007211,0.125109,0.008773,0.002659,0.001556,0.012411
3,0.000021,0.000376,0.000274,0.000276,0.000014,0.002255,0.003132,0.003579,0.000144,0.000192,...,0.001373,0.009667,0.000364,0.008091,0.000967,0.010391,0.003675,0.006302,0.006837,0.005545
4,0.001208,0.000810,0.000083,0.002908,0.000004,0.005848,0.010397,0.007151,0.000333,0.002449,...,0.003531,0.024186,0.001148,0.005435,0.017878,0.028179,0.008829,0.005546,0.001331,0.005982


In [109]:
avg_preds1=preds_df1.mean()
avg_preds1.head()

Product_ID
0972683275    0.000912
1400501466    0.000664
1400501776    0.001063
1400532655    0.001725
140053271X    0.000040
dtype: float64

In [110]:
rmse_df1 = pd.concat([average_rating1, avg_preds1], axis=1)

rmse_df1.columns = ['Avg_actual_ratings1', 'Avg_predicted_ratings1']

rmse_df1.head()

,Avg_actual_ratings1,Avg_predicted_ratings1
Product_ID,,
0972683275,0.003247,0.000912
1400501466,0.003247,0.000664
1400501776,0.003247,0.001063
1400532655,0.005844,0.001725
140053271X,0.000649,0.000040


In [143]:
RMSE1=mean_squared_error(rmse_df1['Avg_actual_ratings1'], rmse_df1['Avg_predicted_ratings1'], squared=False)
print(f'RMSE SVD Model = {RMSE1} \n')
print('sparsity 2',calculate_sparsity(final_ratings_matrix1))

RMSE SVD Model = 0.007341457372919352 

sparsity 2 0.9990339918932599


# dataset 3 Movies :

In [112]:
df2 =pd.read_csv(r"C:\Users\hp\Downloads\Movies_and_TVMerged-20000Sample-importantColumns.csv")

In [113]:
df2

,Product_ID,User_ID,Rating
0,6303542352,A1RXXNP0EXYRI,5.0
1,B00003OSTA,ADNBUKNODH7OX,5.0
2,6303908306,A2G1Z262UMOFQP,5.0
3,B000065V3K,ATJ1TVH5IVPC3,5.0
4,6303398480,A1X4HJ17RZWY9Q,5.0
...,...,...,...
19995,630517511X,A1CDNTB7377YH2,5.0
19996,7799128836,AQ06V0Q6FC4UD,5.0
19997,B00005JMET,A1ZKP9DFOIEBUN,4.0
19998,B0001US8EE,A1MLPCNE4UXDR8,5.0


In [122]:
#la creation de la matrice de rating :
final_ratings_matrix2 = df2.pivot_table(index = 'User_ID', columns ='Product_ID', values = 'Rating').fillna(0)
print('Shape de la matrice_rating est : ', final_ratings_matrix2.shape)

#la recherche des colonnes non nulles dans la matrice rating
num_of_ratings2 = np.count_nonzero(final_ratings_matrix2)
print('num de ratings = ', num_of_ratings2)

#Trouver le nombre possible de notes en fonction du nombre d'utilisateurs et de produits.
possible_num_of_ratings2 = final_ratings_matrix2.shape[0] * final_ratings_matrix2.shape[1]
print('le nombre possible de rating est = ', possible_num_of_ratings2)

#la densité des rating
density2 = (num_of_ratings2/possible_num_of_ratings2)
density2 *= 100
print ('density: {:4.2f}%'.format(density2))

final_ratings_matrix2.head()

Shape de la matrice_rating est :  (19189, 5903)
num de ratings =  19999
le nombre possible de rating est =  113272667
density: 0.02%


Product_ID,0005000009,0005019281,0005119367,0005123968,0307142493,0375810331,0510539610,0578002019,0578047861,0615115187,...,B0006H30F4,B0006H328E,B0006J28E6,B0006N2EZA,B0006OBPTA,B0006OBPTU,B0006OGDCY,B0006OJH0E,B0006Q4AVI,B0006Q94AK
User_ID,,,,,,,,,,,,,,,,,,,,,
A0297640AZ7S0CKL5XZ8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A08161909WK3HU7UYTMW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1009L3XQBX5GJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100V0YLKSBJPV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1018BDG082EVM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
# la transformation : 
final_ratings_sparse2 = csr_matrix(final_ratings_matrix2.values)

In [124]:
# determiner les trois matrices de SVD :
U2, S2, Vt2 = svds(final_ratings_sparse2, k = 3) 

In [125]:
# la transformation de S en une table :
sigma2 = np.diag(S2)

In [127]:
predicted_ratings2 = np.dot(np.dot(U2, sigma2), Vt2) 

# la prédiction :
preds_df2 = pd.DataFrame(abs(predicted_ratings2), columns = final_ratings_matrix2.columns)
preds_matrix2 = csr_matrix(preds_df2.values)

In [130]:
preds_df2.head()

Product_ID,0005000009,0005019281,0005119367,0005123968,0307142493,0375810331,0510539610,0578002019,0578047861,0615115187,...,B0006H30F4,B0006H328E,B0006J28E6,B0006N2EZA,B0006OBPTA,B0006OBPTU,B0006OGDCY,B0006OJH0E,B0006Q4AVI,B0006Q94AK
0,1.415348e-34,1.119153e-34,2.358900e-34,1.080063e-33,1.787664e-34,5.947572e-34,2.873550e-34,3.078357e-34,8.434739e-34,4.493053e-34,...,5.797083e-34,1.528585e-33,1.547813e-33,3.058798e-34,1.226286e-34,9.932254e-35,1.618679e-34,3.998545e-34,5.795319e-35,9.708897e-34
1,6.097266e-36,3.683799e-34,9.872845e-35,1.290253e-34,1.103923e-34,1.646702e-34,2.619708e-34,1.079834e-34,3.433654e-34,1.990619e-34,...,2.248983e-34,2.868463e-34,5.492351e-34,1.253269e-34,1.903726e-34,6.039715e-35,1.047894e-35,6.286794e-34,3.661409e-35,1.188981e-34
2,2.567616e-37,1.328837e-35,4.376915e-36,7.467803e-36,3.655413e-36,5.358785e-36,8.946249e-36,4.725293e-36,1.472773e-35,8.416637e-36,...,9.678611e-36,7.326898e-36,1.626680e-35,5.340330e-36,6.673320e-36,2.008616e-36,3.033449e-38,2.206440e-35,1.210427e-36,6.758825e-36
3,1.108540e-36,3.512175e-34,8.908498e-35,8.864357e-35,1.139348e-34,1.861622e-34,2.651912e-34,9.445791e-35,3.052896e-34,1.782016e-34,...,1.987787e-34,3.359649e-34,5.876504e-34,1.112926e-34,1.894544e-34,6.247207e-35,1.569718e-35,6.255656e-34,3.770694e-35,8.210618e-35
4,1.080743e-36,1.804249e-36,2.336537e-36,9.476575e-36,1.164664e-36,4.208652e-36,1.640052e-36,2.900510e-36,8.099878e-36,4.352846e-36,...,5.520264e-36,1.178671e-35,1.144522e-35,2.927634e-36,4.837145e-37,6.449871e-37,1.336086e-36,1.548849e-36,3.771441e-37,8.502720e-36


In [131]:
recommend_items(100,final_ratings_sparse2,preds_matrix2,10)


la recommandations de user (user_id = 100):

Recommended Products
3524    3.140316e-02
4568    2.425581e-15
5620    9.618277e-16
3918    7.746647e-16
4492    7.503492e-16
86      6.716939e-16
4094    6.084291e-16
5871    5.905042e-16
282     5.833868e-16
5156    5.707262e-16
Name: user_predictions, dtype: float64


In [132]:
final_ratings_matrix2['user_index'] = np.arange(0, final_ratings_matrix2.shape[0])
final_ratings_matrix2.set_index(['user_index'], inplace=True)

# Actual ratings given by users
final_ratings_matrix2.head()

Product_ID,0005000009,0005019281,0005119367,0005123968,0307142493,0375810331,0510539610,0578002019,0578047861,0615115187,...,B0006H30F4,B0006H328E,B0006J28E6,B0006N2EZA,B0006OBPTA,B0006OBPTU,B0006OGDCY,B0006OJH0E,B0006Q4AVI,B0006Q94AK
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
average_rating2 = final_ratings_matrix2.mean()
average_rating2.head()

Product_ID
0005000009    0.000417
0005019281    0.001824
0005119367    0.000261
0005123968    0.000208
0307142493    0.000938
dtype: float64

In [134]:
preds_df.head()

Product_ID,0615179088,0763855553,0842340351,1413313701,1426296355,1600775411,1935341049,3959828160,3959828233,3959828241,...,B01ETNVTSK,B01EU8RG7M,B01F4PYHX6,B01F70HGA4,B01F70HHBC,B01F7RJHIQ,B01H39M7ME,B01HAP3NJM,B01HAP3TX2,B01HAP47PQ
0,7.564024e-33,9.850961e-31,6.917641e-30,2.983428e-23,3.516778e-19,2.633486e-21,1.428447e-30,1.425977e-30,1.331730e-31,3.398348e-31,...,7.266616e-31,4.524394e-32,2.266820e-30,7.904638e-31,1.288809e-30,7.607164e-17,6.493604e-32,2.838636e-31,3.029458e-31,4.550212e-31
1,1.741474e-32,4.314567e-32,1.142237e-31,9.128534e-26,1.076057e-21,8.057879e-24,5.303804e-33,3.724862e-32,3.125698e-33,3.946136e-35,...,1.198471e-33,2.425528e-34,2.307959e-32,3.818906e-33,1.709135e-32,2.300849e-19,1.459446e-33,2.864855e-33,5.324295e-33,3.215412e-33
2,2.975955e-34,1.266463e-33,8.729710e-34,5.396641e-28,6.362213e-24,4.762151e-26,1.804576e-34,6.590794e-34,5.632921e-35,3.190800e-35,...,7.555120e-35,5.043932e-36,1.904615e-34,9.649397e-35,3.461310e-34,1.376681e-21,2.673826e-35,6.229058e-35,6.047173e-35,5.381473e-35
3,3.350959e-33,1.279275e-32,1.239767e-32,2.606500e-27,3.071849e-23,2.302050e-25,1.647228e-33,7.232640e-33,6.142961e-34,2.349593e-34,...,6.275805e-34,5.030608e-35,2.933850e-33,9.276964e-34,3.628910e-33,6.506923e-21,2.902336e-34,6.381016e-34,7.958556e-34,5.704693e-34
4,2.245047e-34,7.535860e-34,1.447895e-33,2.148915e-27,2.533114e-23,1.896777e-25,2.224552e-34,6.007578e-34,5.195672e-35,4.183565e-35,...,9.857537e-35,7.015115e-36,2.267769e-35,1.250849e-34,3.459697e-34,5.490855e-21,2.469125e-35,6.530348e-35,3.031445e-35,7.538190e-35


In [138]:
avg_preds2=preds_df2.mean()
avg_preds2.head()

Product_ID
0005000009    6.693009e-20
0005019281    4.214709e-19
0005119367    1.280886e-19
0005123968    3.500778e-19
0307142493    1.348295e-19
dtype: float64

In [137]:
rmse_df2 = pd.concat([average_rating2, avg_preds2], axis=1)

rmse_df2.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']

rmse_df2.head()

,Avg_actual_ratings,Avg_predicted_ratings
Product_ID,,
0005000009,0.000417,6.693009e-20
0005019281,0.001824,4.214709e-19
0005119367,0.000261,1.280886e-19
0005123968,0.000208,3.500778e-19
0307142493,0.000938,1.348295e-19


In [144]:
RMSE2=mean_squared_error(rmse_df2['Avg_actual_ratings'], rmse_df2['Avg_predicted_ratings'], squared=False)
print(f'RMSE SVD Model = {RMSE2} \n')
print('sparsity 3',calculate_sparsity(final_ratings_matrix2))

RMSE SVD Model = 0.0014051759095625872 

sparsity 3 0.9998234437262786


### ==> une densité plus élevée dans la matrice de notation peut souvent conduire à de meilleures performances pour les modèles de recommandation